#### importing

In [1]:
from kad import KAD
import pandas as pd
from sklearn.model_selection import train_test_split

#### preprocessing

In [2]:
shrooms = pd.read_csv('dataset/mushrooms.csv', usecols=['class', 'cap-shape','cap-surface', 'cap-color', 'gill-size', 'stalk-shape', 'veil-type', 'population'])
shrooms_sample = shrooms.groupby('class').sample(n=250, random_state=123)
shrooms_sample

,class,cap-shape,cap-surface,cap-color,gill-size,stalk-shape,veil-type,population
4831,e,k,y,e,b,e,p,c
605,e,b,s,w,b,e,p,n
6904,e,x,y,c,b,e,p,y
3409,e,f,y,n,b,t,p,v
1661,e,x,f,g,b,t,p,a
...,...,...,...,...,...,...,...,...
2766,p,x,f,g,b,e,p,v
4043,p,f,f,g,b,e,p,y
6579,p,k,y,e,n,t,p,v
4220,p,f,f,y,b,e,p,y


In [ ]:
X = shrooms_sample[shrooms_sample.columns.drop('class')]
y = shrooms_sample[['class']]

In [6]:
X_train, y_train, X_test, y_test = train_test_split(X, y, test_size=0.3, random_state=123, stratify=y)

#### training

In [9]:
kad = KAD(X_train, y_train)
kad.process()

Processing combinations:   0%|          | 0/7 [00:00<?, ?it/s]


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()